In [26]:
#model.save("../models/best_model.keras")  # future-proof .keras format

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import joblib
import cv2

# --------- Define model architecture same as training ---------
def build_model(input_shape=(64, 64, 3), num_classes=35):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', name='conv1'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', name='conv_last'),  # 👈 use this in Grad-CAM
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# --------- Load Model + Weights ---------
model = build_model()
model.load_weights("../models/best_model.h5")
_ = model(np.random.rand(1, 64, 64, 3))  # 👈 this is required to build input/output

print("✅ Model built and weights loaded")

# --------- Load Test Data and Label Encoder ---------
X_test = np.load("../data/processed/X_test.npy")
label_encoder = joblib.load("../logs/label_encoder.pkl")

# --------- Define Grad-CAM Function ---------
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.input],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + 1e-10)
    return heatmap.numpy()

# --------- Overlay Helper ---------
def overlay_heatmap(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), colormap)
    overlayed = cv2.addWeighted(heatmap_colored, alpha, img, 1 - alpha, 0)
    return overlayed

# --------- Pick Image and Generate Heatmap ---------
idx = 5
img = X_test[idx]
img_input = np.expand_dims(img, axis=0)

preds = model.predict(img_input, verbose=0)
pred_class = np.argmax(preds[0])
class_label = label_encoder.inverse_transform([pred_class])[0]

heatmap = make_gradcam_heatmap(img_input, model, last_conv_layer_name="conv_last")
img_uint8 = (img * 255).astype(np.uint8)
overlayed = overlay_heatmap(img_uint8, heatmap)

# --------- Display ---------
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(img_uint8)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(overlayed)
plt.title(f"Grad-CAM: {class_label}")
plt.axis("off")
plt.tight_layout()
plt.show()


✅ Model built and weights loaded


AttributeError: The layer sequential_5 has never been called and thus has no defined input.

In [ ]:
print("Model input:", model.input)
print("Model output:", model.output)
print([layer.name for layer in model.layers])


AttributeError: The layer sequential_2 has never been called and thus has no defined input.

In [31]:
import joblib
joblib.dump(label_encoder, "../logs/label_encoder.pkl")


['../logs/label_encoder.pkl']